<a target="_blank" href="https://colab.research.google.com/github/jingjieyeo/abm_on_colab/blob/main/intro_abm.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Introduction to Agent-Based Modeling with Simple Bacterial Movement

## Introduction

This Google Colab notebook introduces agent-based modeling (ABM) concepts through a simple bacterial movement simulation. Agent-based models let us simulate the behavior of individual entities (agents) and observe how their interactions create emergent patterns at the system level.

In this notebook, we'll:
1. Learn about agent-based modeling and the Mesa framework
2. Create a simple model of bacterial agents that move randomly in an environment
3. Visualize the bacteria's movement and distribution
4. Experiment with different parameters to see how they affect the system

## Setup and Installation

First, let's install the Mesa framework, which we'll use for our agent-based modeling.

In [ ]:
# Install Mesa and required libraries
!pip install mesa matplotlib

# Import required libraries
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import random

from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from IPython.display import clear_output, HTML
from ipywidgets import interact, IntSlider, FloatSlider

## Understanding Agent-Based Modeling

Agent-based modeling (ABM) is a computational approach that simulates the actions and interactions of autonomous agents to assess their effects on the system as a whole. ABMs are used in biology, ecology, social sciences, economics, and many other fields.

Key components of an agent-based model:

1. **Agents**: Individual entities with properties and behaviors
2. **Environment**: The space where agents exist and interact
3. **Rules**: Define how agents behave and interact with each other and the environment
4. **Time**: The simulation progresses in discrete steps

ABMs are particularly useful for studying complex systems where the collective behavior emerges from simple individual rules.

### Why use ABMs for studying bacteria?

Bacteria are excellent candidates for agent-based modeling because:
- They behave as individual entities with simple rules
- Their interactions with the environment and each other lead to complex patterns
- We can observe emergent behaviors like chemotaxis, biofilm formation, and antibiotic resistance
- ABMs let us test hypotheses that would be difficult or time-consuming in a lab

## Implementing a Simple Bacterial Movement Model

In our first model, we'll create a simple representation of bacteria moving randomly in a 2D environment. This models the natural random motion (Brownian motion) that bacteria exhibit when not responding to specific stimuli.

In [ ]:
# Define the bacterial agent
class BacteriumAgent(Agent):
    """
    A simple bacterial agent that moves randomly in the environment.
    """
    def __init__(self, model, speed=1):
        """
        Initialize a new bacterial agent.
        
        Args:
            model: The model the agent belongs to
            speed: How many grid cells the bacterium can move in one step
        """
        
        super().__init__(model)
        self.speed = speed
    
    def move(self):
        """
        Move the bacterium in a random direction.
        """
        # Get possible neighboring cells within the bacterium's speed range
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False, radius=self.speed
        )
        
        # Select a random position to move to
        new_position = self.random.choice(possible_steps)
        
        # Move the bacterium to the new position
        self.model.grid.move_agent(self, new_position)
    
    def step(self):
        """
        Defines what the agent does in one step of the simulation.
        """
        self.move()

In [ ]:
# Define the bacterial model
class BacterialMovementModel(Model):
    """
    A model simulating the random movement of bacteria in a 2D environment.
    """
    def __init__(self, width=50, height=50, n_bacteria=100, speed=1, seed=None):
        """
        Initialize a new bacterial movement model.
        
        Args:
            width: Width of the grid
            height: Height of the grid
            n_bacteria: Initial number of bacteria
            speed: Movement speed of bacteria
            seed: Random seed for reproducibility (optional)
        """
        # Mesa 3.x requires explicit super().__init__() call
        super().__init__(seed=seed)
        
        self.num_bacteria = n_bacteria
        self.grid = MultiGrid(width, height, torus=True)  # Torus=True means the grid wraps around
        
        # Create and place bacteria
        for i in range(self.num_bacteria):
            
            bacterium = BacteriumAgent(self, speed)
            
            # Place the bacterium in a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(bacterium, (x, y))
                    
        # Add data collector to track metrics
        self.datacollector = DataCollector(
            model_reporters={"Grid Occupancy": self.count_grid_occupancy}
        )
    
    def count_grid_occupancy(self):
        """
        Count the percentage of grid cells that contain at least one bacterium.
        """
        occupied_cells = 0
        total_cells = self.grid.width * self.grid.height
        
        for cell_content, (x, y) in self.grid.coord_iter():
            if len(cell_content) > 0:
                occupied_cells += 1
        
        return (occupied_cells / total_cells) * 100
    
    def step(self):
        """
        Advance the model by one step.
        """
        self.datacollector.collect(self)
        
        # This shuffles agents and calls their step() method
        self.agents.shuffle_do("step")

## Visualization

Now let's set up a way to visualize our bacterial movement model in Google Colab.

In [ ]:
# Function to run the model and visualize it
def run_model_with_visualization(width=50, height=50, n_bacteria=100, speed=1, steps=50):
    """
    Run the bacterial movement model and visualize it.
    
    Args:
        width: Width of the grid
        height: Height of the grid
        n_bacteria: Initial number of bacteria
        speed: Movement speed of bacteria
        steps: Number of steps to run the simulation
    """
    
    # Create a new model
    model = BacterialMovementModel(
        width=width, 
        height=height, 
        n_bacteria=n_bacteria, 
        speed=speed
    )
    
    # Store positions at each step for potential animation
    all_positions = []
    
    # Run the model for the specified number of steps
    for i in range(steps):
        model.step()
        
        # Save positions at each step
        positions = [(agent.pos[0], agent.pos[1]) for agent in model.agents]
        all_positions.append(positions)
    
    # Create animation
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_title("Bacterial Movement Animation")
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.grid(True, alpha=0.3)
    
    # Initialize empty scatter plot
    scat = ax.scatter([], [], c='red', s=30, alpha=0.7)
    
    def update(frame):
        positions = all_positions[frame]
        x_positions = [pos[0] for pos in positions]
        y_positions = [pos[1] for pos in positions]
        scat.set_offsets(np.column_stack((x_positions, y_positions)))
        ax.set_title(f"Bacterial Movement (Step {frame+1}/{steps})")
        return scat,
    
    ani = animation.FuncAnimation(fig, update, frames=len(all_positions), 
                                  interval=200, blit=True, repeat=True)
    
    plt.close(fig)  # Close the static plot
    display(HTML(ani.to_jshtml()))
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Final positions scatter plot
    final_positions = all_positions[-1]
    x_positions = [pos[0] for pos in final_positions]
    y_positions = [pos[1] for pos in final_positions]
    
    ax1.scatter(x_positions, y_positions, c='red', s=20, alpha=0.7)
    ax1.set_xlim(0, width)
    ax1.set_ylim(0, height)
    ax1.set_title(f"Final Bacterial Positions (Step {steps})")
    ax1.set_xlabel("X Coordinate")
    ax1.set_ylabel("Y Coordinate")
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Grid occupancy over time
    model_data = model.datacollector.get_model_vars_dataframe()
    ax2.plot(model_data["Grid Occupancy"], 'b-', linewidth=2)
    ax2.set_title("Percentage of Grid Cells Occupied by Bacteria")
    ax2.set_xlabel("Step")
    ax2.set_ylabel("Grid Occupancy (%)")
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return model_data

## Interactive Experiments

Now let's create an interactive interface to experiment with different parameters of our model.

In [ ]:
@interact(
    width=IntSlider(min=20, max=100, step=10, value=50, description='Grid Width:'),
    height=IntSlider(min=20, max=100, step=10, value=50, description='Grid Height:'),
    n_bacteria=IntSlider(min=10, max=500, step=10, value=100, description='Bacteria Count:'),
    speed=IntSlider(min=1, max=5, step=1, value=1, description='Speed:'),
    steps=IntSlider(min=10, max=100, step=10, value=100, description='Steps:')
)
def run_interactive_model(width, height, n_bacteria, speed, steps):
    return run_model_with_visualization(width, height, n_bacteria, speed, steps)

## Extensions and Exercises for Students

Here are some exercises to extend your understanding of agent-based modeling with bacteria:

1. **Modify the movement pattern**: Change the `move()` method in the `BacteriumAgent` class to implement different movement patterns. For example, you could make bacteria prefer to move in the same direction as their last move.

2. **Add energy consumption**: Modify the bacterial agent to consume energy when moving. Bacteria will need to rest or find food to replenish energy.

3. **Add reproduction**: When a bacterium has enough energy, it can reproduce by creating a new bacterium in a neighboring cell using `BacteriumAgent(self.model)`.

4. **Add death**: Bacteria can die when they run out of energy or reach a certain age. Use `self.remove()` to remove agents from the model.

5. **Add an environmental factor**: Create an environmental factor like temperature or pH that affects bacterial movement or survival.

6. **Use AgentSet features**: Experiment with Mesa 3.x's powerful AgentSet functionality:
   - Filter bacteria by energy: `high_energy_bacteria = self.agents.select(lambda a: a.energy > 50)`
   - Group bacteria by type: `grouped = self.agents.groupby("species")`
   - Calculate statistics: `avg_energy = self.agents.agg("energy", func=np.mean)`

## Questions for Reflection

1. What patterns do you observe in the bacterial distribution over time?
2. How does changing the number of bacteria affect the grid occupancy?
3. How does the speed parameter affect the bacteria's spreading pattern?
4. What real-world bacterial behaviors might this simple model represent?
5. What limitations does this model have compared to real bacterial movement?

## Conclusion

In this  notebook, you've learned:
- Basic concepts of agent-based modeling
- How to implement a simple bacterial movement model using Mesa 3.x
- How to visualize and analyze the model's behavior
- How to experiment with different parameters to see their effects

In the next notebook, we'll explore bacterial chemotaxis - how bacteria sense and move toward or away from chemical gradients in their environment.